In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

#Uppgiften kräver att du testar roboten på 4 olika sätt för att se när den är duktigast :

#När den får se siffrorna precis som de är ("Original").

#När vi "krymper" alla siffror till att vara mellan 0 och 1 ("Normaliserad").

#När den får plugga på massor av data (90%) och bara göra prov på lite (10%).

#När den får plugga på lite mindre (t.ex. 70%) och göra prov på mer (30%).

#Och för att vara säker på att roboten inte bara hade tur en gång, måste vi köra varje test 100 gånger och ta snittet (medelvärdet) .

In [14]:
#Laddar in datasetet.
df = pd.read_csv("winequality-red.csv", sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [15]:
# --- PLATSHÅLLARE FÖR DATA (Ta bort detta block och avkommentera raden ovan) ---
# För att koden ska vara körbar här laddar jag en exempelstruktur. 
# Du ska ersätta X och y med dina egna kolumner.
from sklearn.datasets import load_wine
data_raw = load_wine()
df = pd.DataFrame(data_raw.data, columns=data_raw.feature_names)
df['target'] = data_raw.target

In [ ]:
# Dela upp i Features (X) och Target (y)
# Anpassa 'target' till namnet på din målvariabel
target_col = 'target' 
# Dela upp i X (Färg, form, vikt) och y (Äpple/Banan)
X_original = df.drop('target', axis=1) # Allt utom svaret
y = df['target']                       # Bara svaret

In [ ]:
# Skapa normaliserad data (för punkt a.ii)
# Instruktionen tipsar om MinMaxScaler för intervallet [0,1]
scaler = MinMaxScaler() # Skapa verktyget som krymper siffrorna
# Skapa en ny version av X som är normaliserad
X_norm = pd.DataFrame(scaler.fit_transform(X_original), columns=X_original.columns)

In [ ]:
# =============================================================================
# 2. DEFINIERA TEST-KONFIGURATIONER
# =============================================================================
# Vi ska testa kombinationer av:
# a) Dataform: Original vs Normaliserad
# b) Split: 90/10 vs Eget val (här valt till 70/30)

konfigurationer = [
    # Variant 1: Originaldata, 90% träning (0.10 test)
    {"namn": "Original", "data": X_original, "test_size": 0.10, "desc": "Original, 90/10"},
    # Variant 2: Originaldata, 70% träning (0.30 test) - Eget val
    {"namn": "Original", "data": X_original, "test_size": 0.30, "desc": "Original, 70/30 (Eget val)"},
    # Variant 3 & 4: Samma fast med normaliserad data (X_norm)
    {"namn": "Normaliserad", "data": X_norm, "test_size": 0.10, "desc": "Normaliserad, 90/10"},
    {"namn": "Normaliserad", "data": X_norm, "test_size": 0.30, "desc": "Normaliserad, 70/30 (Eget val)"}
]

# Lista för att spara resultaten
alla_resultat = []

print("Startar exekveringar... (Detta körs 100 gånger per konfiguration)\n")

Startar exekveringar... (Detta körs 100 gånger per konfiguration)



In [ ]:
# =============================================================================
# 3. KÖR LOOPAR (100 GÅNGER PER KONFIGURATION) - Enligt punkt b
# =============================================================================
#Det här är kärnan i uppgiften. Instruktionen säger att vi måste köra 100 gånger per inställning och att slumpen ska vara ny varje gång (random_state=None) .
#Förklaring: Vi har två loopar.
#Den yttre loopen (for konf in...) väljer vilken inställning vi testar just nu.
#Den inre loopen (for i in range(100)) upprepar testet 100 gånger.
for konf in konfigurationer: # Loopa genom de 4 varianterna
    accuracies = [] # En tom påse att samla resultaten i (Array)
    best_acc = 0.0
    best_cm = None
    
    # Kör 100 gånger med random_state=None (slumpas nytt varje gång)
    for i in range(100): # Gör detta 100 gånger!
        # Dela upp data
        X_train, X_test, y_train, y_test = train_test_split(
            konf["data"], y, test_size=konf["test_size"], random_state=None
        )
        
        # Träna modell (Decision Tree). Skapa roboten (Beslutsträdet)
        dt = DecisionTreeClassifier(random_state=None)
        #Låt roboten plugga (Träna)
        dt.fit(X_train, y_train)
        
        # Prediktera. Låt roboten göra provet (Prediktera)
        y_pred = dt.predict(X_test)
        
        # Utvärdera
        acc = accuracy_score(y_test, y_pred) ## Rätta provet
        cm = confusion_matrix(y_test, y_pred)
        
        accuracies.append(acc) ## Spara betyget i listan
        
        # Spara den bästa enskilda körningen (för presentation senare)
        if acc > best_acc:
            best_acc = acc
            best_cm = cm
            
    # Beräkna genomsnitt över 100 körningar. Nu är loopen klar. Räkna snittet.
    avg_acc = np.mean(accuracies)
    
    # Spara resultatet. Visa resultatet för denna variant
    resultat_rad = {
        "beskrivning": konf["desc"],
        "split_str": f"{1-konf['test_size']:.2f}/{konf['test_size']:.2f}",
        "avg_acc": avg_acc,
        "best_acc": best_acc,
        "best_cm": best_cm
    }
    alla_resultat.append(resultat_rad)

In [11]:
# =============================================================================
# 4. PRESENTERA RESULTAT (Enligt Figur A2 i Bilaga 2)
# =============================================================================
print("-" * 60)
print("SAMMANSTÄLLNING AV ALLA EXEKVERINGAR (Figur A2-format)")
print("-" * 60)

for res in alla_resultat:
    print(f"Konfiguration: {res['beskrivning']}")
    print(f"Tränings-/Testförhållande: {res['split_str']}")
    print(f"Genomsnittlig noggrannhet över 100 körningar: {res['avg_acc']:.4f}")
    print("Bästa Confusion Matrix:")
    print(res['best_cm'])
    print(f"Bästa accuracy: {res['best_acc']:.4f}")
    print("\n")

------------------------------------------------------------
SAMMANSTÄLLNING AV ALLA EXEKVERINGAR (Figur A2-format)
------------------------------------------------------------
Konfiguration: Original, 90/10
Tränings-/Testförhållande: 0.90/0.10
Genomsnittlig noggrannhet över 100 körningar: 0.9044
Bästa Confusion Matrix:
[[8 0 0]
 [0 5 0]
 [0 0 5]]
Bästa accuracy: 1.0000


Konfiguration: Original, 70/30 (Eget val)
Tränings-/Testförhållande: 0.70/0.30
Genomsnittlig noggrannhet över 100 körningar: 0.9096
Bästa Confusion Matrix:
[[18  0  0]
 [ 0 20  0]
 [ 0  0 16]]
Bästa accuracy: 1.0000


Konfiguration: Normaliserad, 90/10
Tränings-/Testförhållande: 0.90/0.10
Genomsnittlig noggrannhet över 100 körningar: 0.9033
Bästa Confusion Matrix:
[[6 0 0]
 [0 9 0]
 [0 0 3]]
Bästa accuracy: 1.0000


Konfiguration: Normaliserad, 70/30 (Eget val)
Tränings-/Testförhållande: 0.70/0.30
Genomsnittlig noggrannhet över 100 körningar: 0.9117
Bästa Confusion Matrix:
[[18  0  0]
 [ 0 25  0]
 [ 0  0 11]]
Bästa ac

In [ ]:
# =============================================================================
# 5. PRESENTERA DE TRE BÄSTA EXEKVERINGARNA
# =============================================================================
# Sortera baserat på prestanda här använder vi Genomsnittlig Accuracy, Sortera listan så bäst hamnar först.
# men man kan även sortera på 'best_acc' om man vill lyfta potentialen). Vi sorterar efter 'avg_acc' (snittbetyget)
sorterade_resultat = sorted(alla_resultat, key=lambda x: x['avg_acc'], reverse=True)

print("*" * 60)
print("DE TRE BÄSTA KONFIGURATIONERNA (Baserat på genomsnittlig accuracy)")
print("*" * 60)

# Visa topp 3 (eller färre om listan är kortare) eller Visa de 3 översta (Vinnarna)
for i in range(min(3, len(sorterade_resultat))): # Hämta vinnare nummer 1, sen 2, sen 3
    res = sorterade_resultat[i]
    print(f"RANK {i+1}: {res['beskrivning']}")
    print(f"Snitt-accuracy: {res['avg_acc']:.4f}") # Visa snittet (hur stabil den var)
    print(f"Bästa accuracy (i enskild körning): {res['best_acc']:.4f}") # Visa det absolut bästa försöket den gjorde
    print("Confusion Matrix (från bästa körningen):") # Visa Confusion Matrix (krav enligt uppgiften)
    print(res['best_cm'])
    print("-" * 30)

# =============================================================================
# REFLEKTION
# =============================================================================
# Här kan du manuellt skriva ner din reflektion baserat på utskrifterna ovan.
# T.ex: "Skillnaden mellan normaliserad och icke-normaliserad data var..."

************************************************************
DE TRE BÄSTA KONFIGURATIONERNA (Baserat på genomsnittlig accuracy)
************************************************************
RANK 1: Normaliserad, 70/30 (Eget val)
Snitt-accuracy: 0.9117
Bästa accuracy (i enskild körning): 1.0000
Confusion Matrix (från bästa körningen):
[[18  0  0]
 [ 0 25  0]
 [ 0  0 11]]
------------------------------
RANK 2: Original, 70/30 (Eget val)
Snitt-accuracy: 0.9096
Bästa accuracy (i enskild körning): 1.0000
Confusion Matrix (från bästa körningen):
[[18  0  0]
 [ 0 20  0]
 [ 0  0 16]]
------------------------------
RANK 3: Original, 90/10
Snitt-accuracy: 0.9044
Bästa accuracy (i enskild körning): 1.0000
Confusion Matrix (från bästa körningen):
[[8 0 0]
 [0 5 0]
 [0 0 5]]
------------------------------
